In [ ]:
%matplotlib widget

In [ ]:
from glob import glob
import numpy as np
import flammkuchen as fl
from split_dataset import SplitDataset
from bouter import Experiment
import colorspacious
import matplotlib.pyplot as plt

import json
from pathlib import Path

from scipy import stats

from vision_and_navigation.imaging.general import corr2_coeff

In [ ]:
master_ntr = Path(r"\\portulab.synology.me\data\Hagar and Ot\E0040\v10\LS ablation\ntr")
master_control = Path(r"\\portulab.synology.me\data\Hagar and Ot\E0040\v10\LS ablation\control")

pre_list = list(master_ntr.glob("*_ntr_pre*"))
pre_list = [f for f in pre_list if (f / "data_from_suite2p_cells.h5").exists()]

post_list = list(master_ntr.glob("*_ntr_post*"))
post_list = [f for f in post_list if (f / "data_from_suite2p_cells.h5").exists()]


pre_list_ctrl = list(master_control.glob("*_control_pre*"))
pre_list_ctrl = [f for f in pre_list_ctrl if (f / "data_from_suite2p_cells.h5").exists()]

post_list_ctrl = list(master_control.glob("*_control_post*"))
post_list_ctrl = [f for f in post_list_ctrl if (f / "data_from_suite2p_cells.h5").exists()]

num_fish_ntr = len(pre_list)
num_fish_ctrl = len(pre_list)

num_fish_ntr = len(pre_list)

print(num_fish_ntr)

In [ ]:
regions = ['ipn_coords', 'ahb_coords']

In [ ]:
for fish in range(num_fish_ntr):
    
    pre_fish = pre_list[fish]
    exp = Experiment(pre_fish)
    fish_id = exp.fish_id
    print(fish_id)
    
    traces = fl.load(pre_fish / "filtered_traces.h5", "/detr")
    sensory_regressors = fl.load(pre_fish / "sensory_regressors.h5", "/regressors")
    coords_regions = fl.load(pre_fish / "regions_masks_mapzbrain.h5")
    
    # getting region coords (IPN)
    region_idx = coords_regions[regions[0]]
    curr_traces = traces[:, region_idx]

    for direction in range(8):
        current_dir = np.asarray(sensory_regressors.iloc[:, direction])
        if direction is 0:
            reg_corrcoefs = corr2_coeff(curr_traces.T, current_dir.reshape(1, -1)).ravel()
        else:
            reg_corrcoefs = np.append(reg_corrcoefs, corr2_coeff(curr_traces.T, current_dir.reshape(1, -1)).ravel())

    if fish is 0:
        all_regs_corr_pre_ipn = reg_corrcoefs
    else:
        all_regs_corr_pre_ipn = np.append(all_regs_corr_pre_ipn, reg_corrcoefs)
        
    # getting region coords (aHB)
    region_idx = coords_regions[regions[1]]
    curr_traces = traces[:, region_idx]

    for direction in range(8):
        current_dir = np.asarray(sensory_regressors.iloc[:, direction])
        if direction is 0:
            reg_corrcoefs = corr2_coeff(curr_traces.T, current_dir.reshape(1, -1)).ravel()
        else:
            reg_corrcoefs = np.append(reg_corrcoefs, corr2_coeff(curr_traces.T, current_dir.reshape(1, -1)).ravel())

    if fish is 0:
        all_regs_corr_pre_ahb = reg_corrcoefs
    else:
        all_regs_corr_pre_ahb = np.append(all_regs_corr_pre_ahb, reg_corrcoefs)
                
    #######################################
    post_fish = post_list[fish]
    
    exp = Experiment(post_fish)
    fish_id = exp.fish_id
    print(fish_id)

    traces = fl.load(post_fish / "filtered_traces.h5", "/detr")
    sensory_regressors = fl.load(post_fish / "sensory_regressors.h5", "/regressors")
    coords_regions = fl.load(post_fish / "regions_masks_mapzbrain.h5")
    
    # getting region coords (IPN)
    region_idx = coords_regions[regions[0]]
    curr_traces = traces[:, region_idx]

    for direction in range(8):
        current_dir = np.asarray(sensory_regressors.iloc[:, direction])
        if direction is 0:
            reg_corrcoefs = corr2_coeff(curr_traces.T, current_dir.reshape(1, -1)).ravel()
        else:
            reg_corrcoefs = np.append(reg_corrcoefs, corr2_coeff(curr_traces.T, current_dir.reshape(1, -1)).ravel())

    if fish is 0:
        all_regs_corr_post_ipn = reg_corrcoefs
    else:
        all_regs_corr_post_ipn = np.append(all_regs_corr_post_ipn, reg_corrcoefs)
        
    # getting region coords (aHB)
    region_idx = coords_regions[regions[1]]
    curr_traces = traces[:, region_idx]

    for direction in range(8):
        current_dir = np.asarray(sensory_regressors.iloc[:, direction])
        if direction is 0:
            reg_corrcoefs = corr2_coeff(curr_traces.T, current_dir.reshape(1, -1)).ravel()
        else:
            reg_corrcoefs = np.append(reg_corrcoefs, corr2_coeff(curr_traces.T, current_dir.reshape(1, -1)).ravel())

    if fish is 0:
        all_regs_corr_post_ahb = reg_corrcoefs
    else:
        all_regs_corr_post_ahb = np.append(all_regs_corr_post_ahb, reg_corrcoefs)

In [ ]:
sessions = ['pre', 'post']
ntr_roi_source = {session:{region:[] for region in regions} for session in sessions}
n_directions = 8

for session, fish_list in zip(sessions, [pre_list, post_list]):
    for fish in range(num_fish_ntr):
        pre_fish = fish_list[fish]
        coords_regions = fl.load(pre_fish / "regions_masks_mapzbrain.h5")
        for region in regions:
            ntr_roi_source[session][region].append(np.full(len(coords_regions[region]*n_directions), fish))
            
for session in sessions:
    for region in regions:
        ntr_roi_source[session][region] = np.concatenate(ntr_roi_source[session][region])

In [ ]:
for fish in range(num_fish_ctrl):
    
    pre_fish = pre_list_ctrl[fish]
    exp = Experiment(pre_fish)
    fish_id = exp.fish_id
    print(fish_id)
    
    traces = fl.load(pre_fish / "filtered_traces.h5", "/detr")
    sensory_regressors = fl.load(pre_fish / "sensory_regressors.h5", "/regressors")
    coords_regions = fl.load(pre_fish / "regions_masks_mapzbrain.h5")
    
        
    # getting region coords (IPN)
    region_idx = coords_regions[regions[0]]
    curr_traces = traces[:, region_idx]

    for direction in range(8):
        current_dir = np.asarray(sensory_regressors.iloc[:, direction])
        if direction is 0:
            print('direction is 0')
            reg_corrcoefs = corr2_coeff(curr_traces.T, current_dir.reshape(1, -1)).ravel()
        else:
            reg_corrcoefs = np.append(reg_corrcoefs, corr2_coeff(curr_traces.T, current_dir.reshape(1, -1)).ravel())

    if fish is 0:
        all_regs_corr_pre_ipn_ctrl = reg_corrcoefs
    else:
        all_regs_corr_pre_ipn_ctrl = np.append(all_regs_corr_pre_ipn_ctrl, reg_corrcoefs)
        
    # getting region coords (aHB)
    region_idx = coords_regions[regions[1]]
    curr_traces = traces[:, region_idx]

    for direction in range(8):
        current_dir = np.asarray(sensory_regressors.iloc[:, direction])
        if direction is 0:
            reg_corrcoefs = corr2_coeff(curr_traces.T, current_dir.reshape(1, -1)).ravel()
        else:
            reg_corrcoefs = np.append(reg_corrcoefs, corr2_coeff(curr_traces.T, current_dir.reshape(1, -1)).ravel())

    if fish is 0:
        all_regs_corr_pre_ahb_ctrl = reg_corrcoefs
    else:
        all_regs_corr_pre_ahb_ctrl = np.append(all_regs_corr_pre_ahb_ctrl, reg_corrcoefs)
                
    #######################################
    post_fish = post_list_ctrl[fish]
    
    exp = Experiment(post_fish)
    fish_id = exp.fish_id
    print(fish_id)

    traces = fl.load(post_fish / "filtered_traces.h5", "/detr")
    sensory_regressors = fl.load(post_fish / "sensory_regressors.h5", "/regressors")
    coords_regions = fl.load(post_fish / "regions_masks_mapzbrain.h5")
    
    # getting region coords (IPN)
    region_idx = coords_regions[regions[0]]
    curr_traces = traces[:, region_idx]

    for direction in range(8):
        current_dir = np.asarray(sensory_regressors.iloc[:, direction])
        if direction is 0:
            print('direction is 0')
            reg_corrcoefs = corr2_coeff(curr_traces.T, current_dir.reshape(1, -1)).ravel()
        else:
            reg_corrcoefs = np.append(reg_corrcoefs, corr2_coeff(curr_traces.T, current_dir.reshape(1, -1)).ravel())

    if fish is 0:
        all_regs_corr_post_ipn_ctrl = reg_corrcoefs
    else:
        all_regs_corr_post_ipn_ctrl = np.append(all_regs_corr_post_ipn_ctrl, reg_corrcoefs)
        
    # getting region coords (aHB)
    region_idx = coords_regions[regions[1]]
    curr_traces = traces[:, region_idx]

    for direction in range(8):
        current_dir = np.asarray(sensory_regressors.iloc[:, direction])
        if direction is 0:
            reg_corrcoefs = corr2_coeff(curr_traces.T, current_dir.reshape(1, -1)).ravel()
        else:
            reg_corrcoefs = np.append(reg_corrcoefs, corr2_coeff(curr_traces.T, current_dir.reshape(1, -1)).ravel())

    if fish is 0:
        all_regs_corr_post_ahb_ctrl = reg_corrcoefs
    else:
        all_regs_corr_post_ahb_ctrl = np.append(all_regs_corr_post_ahb_ctrl, reg_corrcoefs)

In [ ]:
sessions = ['pre', 'post']
control_roi_source = {session:{region:[] for region in regions} for session in sessions}
n_directions = 8

for session, fish_list in zip(sessions, [pre_list_ctrl, post_list_ctrl]):
    for fish in range(num_fish_ctrl):
        pre_fish = fish_list[fish]
        coords_regions = fl.load(pre_fish / "regions_masks_mapzbrain.h5")
        for region in regions:
            control_roi_source[session][region].append(np.full(len(coords_regions[region]*n_directions), fish))
            
for session in sessions:
    for region in regions:
        control_roi_source[session][region] = np.concatenate(control_roi_source[session][region])

In [ ]:
fig2, ax2 = plt.subplots(2,2, figsize=(5,4), sharex=True)

ax2[0,0].hist(all_regs_corr_pre_ipn, bins=100, alpha=1, density=True, histtype='step', color='cornflowerblue')
ax2[0,0].hist(all_regs_corr_post_ipn, bins=100, alpha=1, density=True, histtype='step', color='mediumorchid')
ax2[0,0].spines['right'].set_visible(False)
ax2[0,0].spines['top'].set_visible(False)

ax2[0,1].hist(all_regs_corr_pre_ahb, bins=100, alpha=1, density=True, histtype='step', color='cornflowerblue')
ax2[0,1].hist(all_regs_corr_post_ahb, bins=100, alpha=1, density=True, histtype='step', color='mediumorchid')
ax2[0,1].spines['right'].set_visible(False)
ax2[0,1].spines['top'].set_visible(False)
ax2[0,0].set_ylabel('Density (IPN)')
ax2[0,1].set_ylabel('Density (aHB)')


ax2[0,0].set_xlim(-0.7, 0.7)

In [ ]:
ax2[1,0].hist(all_regs_corr_pre_ipn_ctrl, bins=100, alpha=1, density=True, histtype='step', color='cornflowerblue')
ax2[1,0].hist(all_regs_corr_post_ipn_ctrl, bins=100, alpha=1, density=True, histtype='step', color='mediumorchid')
ax2[1,0].spines['right'].set_visible(False)
ax2[1,0].spines['top'].set_visible(False)

ax2[1,1].hist(all_regs_corr_pre_ahb_ctrl, bins=100, alpha=1, density=True, histtype='step', color='cornflowerblue')
ax2[1,1].hist(all_regs_corr_post_ahb_ctrl, bins=100, alpha=1, density=True, histtype='step', color='mediumorchid')
ax2[1,1].spines['right'].set_visible(False)
ax2[1,1].spines['top'].set_visible(False)
ax2[1,0].set_ylabel('Density (IPN)')
ax2[1,1].set_ylabel('Density (aHB)')

In [ ]:
fig2.subplots_adjust(left=0.15, right=0.95, wspace=0.6, hspace=0.1)

## New plots

In [ ]:
n_fish = 11
n_bins = 100
hist_lims = [-.5, .5]

colors = ['cornflowerblue', 'mediumorchid']

In [ ]:
ntr_data_dict = {'pre':{'ipn_coords':all_regs_corr_pre_ipn,
                        'ahb_coords': all_regs_corr_pre_ahb},
                 'post':{'ipn_coords':all_regs_corr_post_ipn,
                        'ahb_coords': all_regs_corr_post_ahb}}

control_data_dict = {'pre':{'ipn_coords':all_regs_corr_pre_ipn_ctrl,
                        'ahb_coords': all_regs_corr_pre_ahb_ctrl},
                     'post':{'ipn_coords':all_regs_corr_post_ipn_ctrl,
                            'ahb_coords': all_regs_corr_post_ahb_ctrl}}

In [ ]:
fig, axes = plt.subplots(2,2, figsize=(6,6))

for row, (dataset, roi_source_dict) in enumerate(zip([ntr_data_dict, control_data_dict], [ntr_roi_source, control_roi_source])):
    for i_session, session in enumerate(sessions):
        for i_region, region in enumerate(regions):

            fish_hist = np.empty((n_fish, n_bins))
            bins = np.empty((n_fish, n_bins+1))

            for fish in range(n_fish):
                sel_vals = dataset[session][region][roi_source_dict[session][region] == fish]
                fish_hist[fish, :], bins[fish, :] = np.histogram(sel_vals, bins=n_bins, range=hist_lims, density=True)

            axes[row, i_region].stairs(np.mean(fish_hist, 0), bins[0], color=colors[i_session], linewidth=1.5)
            axes[row, i_region].set_title('{}'.format(region))
            
            x = (bins[0, 1:] + bins[0, :-1]) / 2
            axes[row, i_region].fill_between(x, np.mean(fish_hist, 0)-np.std(fish_hist, 0),np.mean(fish_hist, 0)+np.std(fish_hist, 0), color=colors[i_session], alpha=.25)
            
for ax in axes.ravel():
    ax.set_xticks([-.5, 0, .5])
    ax.set_xlabel('Correlation')
    
for ax, treatment, in zip(axes[:, 0], ['NTR', 'Control']):
    ax.set_ylabel('{}'.format(treatment))
    
plt.tight_layout()

In [ ]:
directions = ['right', 'backward right', 'backward', 'backward left', 'left', 'forward left', 'forward', 'forward right']
    
for direction in range(n_directions):
    fig, axes = plt.subplots(2,2, figsize=(6,6))

    for row, (dataset, roi_source_dict) in enumerate(zip([ntr_data_dict, control_data_dict], [ntr_roi_source, control_roi_source])):
        for i_session, session in enumerate(sessions):
            for i_region, region in enumerate(regions):

                fish_hist = np.empty((n_fish, n_bins))
                bins = np.empty((n_fish, n_bins+1))

                for fish in range(n_fish):
                    sel_vals = dataset[session][region][roi_source_dict[session][region] == fish][direction::n_directions]
                    fish_hist[fish, :], bins[fish, :] = np.histogram(sel_vals, bins=n_bins, range=hist_lims, density=True)

                axes[row, i_region].stairs(np.mean(fish_hist, 0), bins[0], color=colors[i_session], linewidth=1.5)
                axes[row, i_region].set_title('{}'.format(region))

                x = (bins[0, 1:] + bins[0, :-1]) / 2
                axes[row, i_region].fill_between(x, np.mean(fish_hist, 0)-np.std(fish_hist, 0),np.mean(fish_hist, 0)+np.std(fish_hist, 0), color=colors[i_session], alpha=.25)

            
    for ax in axes.ravel():
        ax.set_xticks([-.5, 0, .5])
        ax.set_xlabel('Correlation')

    for ax, treatment, in zip(axes[:, 0], ['NTR', 'Control']):
        ax.set_ylabel('{}'.format(treatment))

    plt.suptitle('Direction: {}'.format(directions[direction]))
    plt.tight_layout()
    
    fig.savefig(fig_path / 'distribution of correlation values only contours with_std_{}.pdf'.format(directions[direction].upper()), dpi=300)

In [ ]:
fig, axes = plt.subplots(2,2, figsize=(6,6))

for row, (dataset, roi_source_dict) in enumerate(zip([ntr_data_dict, control_data_dict], [ntr_roi_source, control_roi_source])):
    for i_session, session in enumerate(sessions):
        for i_region, region in enumerate(regions):

            fish_hist = np.empty((n_fish, n_bins))
            bins = np.empty((n_fish, n_bins+1))

            for fish in range(n_fish):
                sel_vals = dataset[session][region][roi_source_dict[session][region] == fish][direction::n_directions]
                fish_hist[fish, :], bins[fish, :] = np.histogram(sel_vals, bins=n_bins, range=hist_lims, density=True)

            axes[row, i_region].stairs(np.mean(fish_hist, 0), bins[0], color=colors[i_session], linewidth=1.5)
            axes[row, i_region].set_title('{}'.format(region))
            
            x = (bins[0, 1:] + bins[0, :-1]) / 2
            axes[row, i_region].fill_between(x, np.mean(fish_hist, 0)-np.std(fish_hist, 0),np.mean(fish_hist, 0)+np.std(fish_hist, 0), color=colors[i_session], alpha=.25)
            

In [ ]:
# Calculate the F-statistic
f = np.var(all_regs_corr_pre_ipn_ctrl, ddof=1) / np.var(all_regs_corr_post_ipn_ctrl, ddof=1)

# Calculate degrees of freedom
dfn = len(all_regs_corr_pre_ipn_ctrl) - 1  # degrees of freedom numerator
dfd = len(all_regs_corr_post_ipn_ctrl) - 1  # degrees of freedom denominator

# Perform the F-test
p_value = stats.f.cdf(f, dfn, dfd)
p_value = 2 * min(p_value, 1 - p_value)  # two-tailed test

print(f"F-statistic: {f}")
print(f"p-value: {p_value}")

In [ ]:
# Calculate the F-statistic
f = np.var(all_regs_corr_pre_ahb_ctrl, ddof=1) / np.var(all_regs_corr_post_ahb_ctrl, ddof=1)

# Calculate degrees of freedom
dfn = len(all_regs_corr_pre_ahb_ctrl) - 1  # degrees of freedom numerator
dfd = len(all_regs_corr_post_ahb_ctrl) - 1  # degrees of freedom denominator

# Perform the F-test
p_value = stats.f.cdf(f, dfn, dfd)
p_value = 2 * min(p_value, 1 - p_value)  # two-tailed test

print(f"F-statistic: {f}")
print(f"p-value: {p_value}")

In [ ]:
# Calculate the F-statistic
f = np.var(all_regs_corr_pre_ipn, ddof=1) / np.var(all_regs_corr_post_ipn, ddof=1)

# Calculate degrees of freedom
dfn = len(all_regs_corr_pre_ipn) - 1  # degrees of freedom numerator
dfd = len(all_regs_corr_post_ipn) - 1  # degrees of freedom denominator

# Perform the F-test
p_value = stats.f.cdf(f, dfn, dfd)
p_value = 2 * min(p_value, 1 - p_value)  # two-tailed test

print(f"F-statistic: {f}")
print(f"p-value: {p_value}")

In [ ]:
# Calculate the F-statistic
f = np.var(all_regs_corr_pre_ahb, ddof=1) / np.var(all_regs_corr_post_ahb, ddof=1)

# Calculate degrees of freedom
dfn = len(all_regs_corr_pre_ahb) - 1  # degrees of freedom numerator
dfd = len(all_regs_corr_post_ahb) - 1  # degrees of freedom denominator

# Perform the F-test
p_value = stats.f.cdf(f, dfn, dfd)
p_value = 2 * min(p_value, 1 - p_value)  # two-tailed test

print(f"F-statistic: {f}")
print(f"p-value: {p_value}")